# Simulation of an Electrical System based on Thermoelectrics Plants

In [1]:
from src.map import Map2D, GraphMap
from src.circuits import Circuit, Block
from src.thermoelectrics import Thermoelectric
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from src.simulation_constants import (
    NO_CIRCUITS,
    NO_THERMOELECTRICS,
    IMPORTANCE_ALPHA,
    RANDOM,
    DISTANCE_REGULATOR
)


## Initialize bases 

In [2]:
map_2d = Map2D(
    no_circuits=NO_CIRCUITS,
    no_thermoelectrics=NO_THERMOELECTRICS,
)


map_2d.visualize()

In [3]:
graphMap = GraphMap(
    thermoelectric_labels=[f"Th{i}" for i in range(NO_THERMOELECTRICS)],
    circuits_labels=[f"C{i}" for i in range(NO_CIRCUITS)],
    towers_labels=[f"Tw{i}" for i in range(len(map_2d.towers_positions))],
    thermoelectrics_positions=map_2d.thermoelectrics_positions,
    circuits_positions=map_2d.circuits_positions,
    towers_positions=map_2d.towers_positions,
)

graphMap.visualize()

In [4]:
distance_cost_template = graphMap.thermoelectric_generation_cost

# Generate circuits for map

electric_system_circuits_list: list[Circuit] = []
for i in range(NO_CIRCUITS):
    electric_system_circuits_list.append(Circuit(graphMap.circuits_nodes[i].id))

# Generate Thermoelectric generation based in the nearest circuits
mapper_circuit_with_thermoelectric = {}

for c in electric_system_circuits_list:

    filtered = [f for f in graphMap.thermoelectric_generation_cost if f[1] == c.id]
    filtered = sorted(filtered, key=lambda x: x[2])
    mapper_circuit_with_thermoelectric[c.id] = filtered[0][0]


# Create edges for the plotly graph
edges = []
edges_labels = []
for circuit, thermoelectric in mapper_circuit_with_thermoelectric.items():

    circuitPos = [c.position for c in graphMap.circuits_nodes if c.id == circuit][0]
    thermoelectricPos = [
        t.position for t in graphMap.thermoelectrics_nodes if t.id == thermoelectric
    ][0]

    edges.append((circuitPos, thermoelectricPos))
    edges_labels.append(f"{circuit} -> {thermoelectric}")


# Create a plotly graph
fig = go.Figure()

# Add edges to the plotly graph
for i, edge in enumerate(edges):
    circuit, thermoelectric = edge
    fig.add_trace(
        go.Scatter(
            x=[circuit[0], thermoelectric[0]],
            y=[circuit[1], thermoelectric[1]],
            mode="lines",
            line=dict(color="black"),
            name="Wire Connection",
            showlegend=False,
            hoverinfo="text",
            text=edges_labels[i],
            hoverlabel=dict(bgcolor="white", font_size=16, font_family="Rockwell"),
        )
    )

# Add nodes to the plotly graph
all_nodes = graphMap.circuits_nodes + graphMap.thermoelectrics_nodes
for i, node in enumerate(all_nodes):
    fig.add_trace(
        go.Scatter(
            x=[node.position[0]],
            y=[node.position[1]],
            mode="markers+text",
            marker=dict(
                size=10, color="blue" if i >= len(graphMap.circuits_nodes) else "red"
            ),
            text=[node.id],
            textposition="top center",
            hoverinfo="text",
        )
    )

fig.update_layout(
    showlegend=False,
    title="Circuits and Thermoelectrics",
    height=800,
    plot_bgcolor="lightgreen",
)

fig.show()

In [5]:
# Generate the distance matrix


def distance_template_to_distance_matrix(
    template: list[tuple[str, str, float, list[str]]],
    thermoelectrics: list[str],
    circuits: list[str],
):
    matrix = [[-1 for _ in range(len(circuits))] for _ in range(len(thermoelectrics))]

    c_map = {}
    t_map = {}

    for i, t in enumerate(thermoelectrics):
        t_map[t] = i

    for i, c in enumerate(circuits):
        c_map[c] = i

    max_cost = 1
    for t, c, cost, _ in template:
        matrix[t_map[t]][c_map[c]] = cost
        max_cost = max(max_cost, cost)

    for t, c, _, _ in template:
        matrix[t_map[t]][c_map[c]] /= max_cost * DISTANCE_REGULATOR

    return matrix


matrix = distance_template_to_distance_matrix(
    distance_cost_template,
    [t.id for t in graphMap.thermoelectrics_nodes],
    [c.id for c in graphMap.circuits_nodes],
)


distance_matrix_df = pd.DataFrame(
    matrix,
    index=[t.id for t in graphMap.thermoelectrics_nodes],
    columns=[c.id for c in graphMap.circuits_nodes],
)

distance_matrix_df

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49
Th0,0.004552,0.008274,0.008962,0.003300,0.003707,0.000352,0.001133,0.001885,0.000370,0.000494,...,0.008982,0.000929,0.000104,0.002405,0.006930,0.005746,0.000247,0.007013,0.000221,0.003826
Th1,0.002903,0.000986,0.001673,0.004331,0.003903,0.007742,0.008524,0.005691,0.007330,0.007885,...,0.001693,0.008320,0.007495,0.005363,0.000755,0.001785,0.007638,0.000664,0.007612,0.003653
Th2,0.003189,0.006912,0.007600,0.001938,0.002344,0.001763,0.002544,0.000523,0.001351,0.001905,...,0.007619,0.002340,0.001515,0.001042,0.005567,0.004384,0.001658,0.005650,0.001632,0.002463
Th3,0.000188,0.003911,0.004598,0.001436,0.001009,0.004848,0.005629,0.002796,0.004436,0.004990,...,0.004618,0.005426,0.004600,0.002469,0.002566,0.001382,0.004744,0.002649,0.004717,0.000759
Th4,0.005570,0.009293,0.009980,0.004319,0.004725,0.001090,0.000271,0.002904,0.001389,0.001024,...,0.010000,0.000110,0.001122,0.003423,0.007948,0.006765,0.001266,0.008031,0.001066,0.004844
Th5,0.002894,0.006617,0.007304,0.001642,0.002049,0.002402,0.003183,0.000350,0.001990,0.002544,...,0.007324,0.002979,0.002154,0.000747,0.005272,0.004088,0.002297,0.005355,0.002271,0.002168
Th6,0.003621,0.000175,0.000863,0.005048,0.004621,0.008460,0.009241,0.006408,0.008048,0.008603,...,0.000882,0.009038,0.008212,0.006081,0.001473,0.002502,0.008356,0.001381,0.008329,0.004371
Th7,0.000955,0.004677,0.005365,0.000537,0.000110,0.003949,0.004730,0.001897,0.003537,0.004091,...,0.005384,0.004527,0.003701,0.001570,0.003332,0.002149,0.003845,0.003415,0.003818,0.000229


In [6]:
# Generate thermoelectrics
electric_system_thermoelectrics_list: list[Thermoelectric] = []

for i, t in enumerate(graphMap.thermoelectrics_nodes):
    generated_thermoelectric_min_cost = 0
    real_cost = 0
    for j, c in enumerate(graphMap.circuits_nodes):
        if mapper_circuit_with_thermoelectric[c.id] == t.id:
            generated_thermoelectric_min_cost += (
                electric_system_circuits_list[j].mock_electric_consume * (matrix[i][j] + 1))
            real_cost += sum([sum(x.demand_per_hour) for x in  electric_system_circuits_list[j].blocks])

    # print(f"Thermoelectric {t.id} has a total capacity of {generated_thermoelectric_min_cost}, but we add 1000 to the total capacity, and real cost is {real_cost}")
    
    electric_system_thermoelectrics_list.append(
        Thermoelectric(
            id=t.id,
            total_capacity=generated_thermoelectric_min_cost,
        )
    )
    thermoelectric_data = {
        "ID": [t.id for t in electric_system_thermoelectrics_list],
        "Total Capacity": [
            t.total_capacity for t in electric_system_thermoelectrics_list
        ],
    }

    thermoelectric_df = pd.DataFrame(thermoelectric_data)


sorted_electric_system_circuit_list = sorted(
    electric_system_circuits_list,
    key=lambda x: list(
        filter(
            lambda y: mapper_circuit_with_thermoelectric[x.id] == y.id,
            graphMap.thermoelectrics_nodes,
        )
    )[0].id,
)

colors = [
    list(
        filter(
            lambda x, y=circuit: mapper_circuit_with_thermoelectric[y.id] == x.id,
            graphMap.thermoelectrics_nodes,
        )
    )[0].id
    for circuit in sorted_electric_system_circuit_list
]


fig = px.bar(
    thermoelectric_df,
    x="ID",
    y="Total Capacity",
    title="Total Capacity of Thermoelectrics",
    height=800,
    color=[t.id for t in electric_system_thermoelectrics_list],
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig.show()

In [7]:

mock_electric_consume_data = {
    "Circuit ID": [c.id for c in sorted_electric_system_circuit_list],
    "Mock Electric Consume": [c.mock_electric_consume for c in sorted_electric_system_circuit_list],
}

mock_electric_consume_df = pd.DataFrame(mock_electric_consume_data)

fig = px.bar(
    mock_electric_consume_df,
    x="Circuit ID",
    y="Mock Electric Consume",
    title="Electric Consume per Circuit [24 hours]",
    height=800,
    color=colors
)
fig.show()

In [8]:
max_population_of_circuits = -1
max_population_of_block = -1

for circuit in electric_system_circuits_list:
    max_population_of_circuits = max(
        circuit.get_all_block_population(), max_population_of_circuits
    )
    for block in circuit.blocks:
        max_population_of_block = max(block.citizens.amount, max_population_of_block)

auxiliary_data_max_population_of_circuits = max_population_of_circuits
auxiliary_data_max_population_of_block = max_population_of_block


def get_circuit_importance(circuit: Circuit) -> float:
    return (
        circuit.get_all_block_population() / auxiliary_data_max_population_of_circuits
    ) * IMPORTANCE_ALPHA + circuit.industrialization * (1 - IMPORTANCE_ALPHA)


def get_block_importance(block: Block) -> float:
    return (
        block.citizens.amount / auxiliary_data_max_population_of_circuits
    ) * IMPORTANCE_ALPHA + block.industrialization * (1 - IMPORTANCE_ALPHA)


def set_importance(ci: list[Circuit]):
    for circuit in ci:
        for block in circuit.blocks:
            block.importance = get_block_importance(block)
        circuit.importance = get_circuit_importance(circuit)

set_importance(electric_system_circuits_list)

max_population_data = {
    "Max Population of Circuits": [auxiliary_data_max_population_of_circuits],
    "Max Population of Blocks": [auxiliary_data_max_population_of_block],
}

max_population_df = pd.DataFrame(max_population_data)
# TODO : update plotly
max_population_df

,Max Population of Circuits,Max Population of Blocks
0,19689,4985


In [9]:
# All data of one block of a circuit (importance, citizens, industrialization, demand per hour)

circuit_stats = electric_system_circuits_list[0]

blocks = circuit_stats.blocks

citizens = []
importance = []
demand_per_hour = []

for block in blocks:
    citizens.append(block.citizens.amount)
    importance.append(block.importance)
    demand_per_hour.append(sum(block.predicted_demand_per_hour))

block_stats_df = pd.DataFrame(
    {
        "Citizens": citizens,
        "Importance": importance,
        "Demand per Hour": demand_per_hour,
    }
)

fig_citizens = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Citizens",
    title=f"Block Citizens in Circuit {circuit_stats.id}",
    height=800,
)

fig_importance = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Importance",
    title=f"Block Importance in Circuit {circuit_stats.id}",
    height=800,
)

fig_demand_per_hour = px.bar(
    block_stats_df,
    x=block_stats_df.index,
    y="Demand per Hour",
    title=f"Block Demand per Hour in Circuit {circuit_stats.id}",
    height=800,
)


fig_citizens.show()
fig_importance.show()
fig_demand_per_hour.show()

In [10]:
# Compare data from circuits

citizens = []
importance = []
demand_per_hour = []

for circuit in sorted_electric_system_circuit_list:
    
    importance.append(circuit.importance)
    demand_per_hour.append(
        sum([sum(block.predicted_demand_per_hour) for block in circuit.blocks])
    )

# Separate for properties
citizens = [
    sum(block.citizens.amount for block in circuit.blocks)
    for circuit in electric_system_circuits_list
]

circuit_comparison_df = pd.DataFrame(
    {   "Circuit ID": [c.id for c in sorted_electric_system_circuit_list], 
        "Citizens": citizens,
        "Importance": importance,
        "Demand per Hour": demand_per_hour,
    }
)

fig_citizens = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Citizens",
    title="Total Citizens per Circuit",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)

fig_importance = px.bar(
    circuit_comparison_df,
    x="Circuit ID",
    y="Importance",
    title="Importance per Circuit",
    height=800,
    color=colors,
    color_discrete_sequence=px.colors.qualitative.Plotly,
    labels={"color": "Thermoelectric ID"},
)



fig_citizens.show()
fig_importance.show()


In [11]:
circuit_id = "C0"
circuit = next(c for c in electric_system_circuits_list if c.id == circuit_id)

block_demand_data = {"Block ID": [], "Hour": [], "Demand": []}

for i, block in enumerate(circuit.blocks):
    for hour, demand in enumerate(block.predicted_demand_per_hour):
        block_demand_data["Block ID"].append(f"Block {i + 1}")
        block_demand_data["Hour"].append(hour)
        block_demand_data["Demand"].append(demand)

block_demand_df = pd.DataFrame(block_demand_data)

fig_block_demand = px.line(
    block_demand_df,
    x="Hour",
    y="Demand",
    color="Block ID",
    title=f"Hourly Demand per Block in Circuit {circuit_id}",
    labels={"Demand": "Demand (kWh)", "Hour": "Hour"},
    height=800
)

fig_block_demand.show()

circuit_demand_data = {
    "Hour": list(range(24)),
    "Demand": [
        sum(block.predicted_demand_per_hour[hour] for block in circuit.blocks)
        for hour in range(24)
    ],
}

circuit_demand_df = pd.DataFrame(circuit_demand_data)

fig_circuit_demand = px.line(
    circuit_demand_df,
    x="Hour",
    y="Demand",
    title=f"Hourly Demand of Circuit {circuit_id}",
    labels={"Demand": "Demand (kWh)", "Hour": "Hour"},
    height=800,
)

fig_circuit_demand.show()

# Simulation

In [12]:
from src.people import (
    ThermoelectricAgent,
    ThermoelectricAgentPerception,
    ThermoelectricAgentAction,
    ChiefElectricCompanyAgent,
    ChiefElectricCompanyAgentPerception,
    ChiefElectricCompanyAction,
)
from src.worldstate import WorldState
import time
from src.bdi import (
    CECAGeneratedDesire,
    CECAMaxStoredEnergyDesire,
    CECAMeetDemandDesire,
    CECAPrioritizeBlockImportance,
    CECAPrioritizeBlockOpinion,
    CECAPrioritizeConsecutiveDaysOff,
    CECAPrioritizeDaysOff,
    TAMaxPowerOutputDesire,
    TAMeetEnergyDemandDesire,
    TAMinimizeDowntimeDesire,
    TAPreventUnexpectedBreakdownDesire,
    TAPrioritizeCriticalPartsRepairDesire,
    TARepairPartsDesire,
)


class Simulation:
    def __init__(
        self,
        circuits: list[Circuit],
        thermoelectrics: list[Thermoelectric],
        distance_matrix: list[list[float]],
        get_circuit_importance: callable,
        get_block_importance: callable,
    ):
        self.world_state_manager = WorldState(
            circuits=circuits,
            thermoelectrics=thermoelectrics,
            distance_matrix=distance_matrix,
            get_block_importance=get_block_importance,
            get_circuit_importance=get_circuit_importance,
        )

        self.thermoelectric_agents: list["ThermoelectricAgent"] = []
        self.thermoelectric_agent_actions: list[list["ThermoelectricAgentAction"]] = []
        self.chief_agent = None
        self.chief_agent_actions: list["ChiefElectricCompanyAction"] = []

        # STATS
        self.general_deficit_stats: list[float] = []
        self.general_consume_stats: list[float] = []
        self.general_demand_stats: list[float] = []
        self.general_opinion_stats: list[float] = []

        self.circuits_stats_daily_deficit: list[list[float]] = []
        self.circuits_stats_daily_demand: list[list[float]] = []
        self.circuits_stats_daily_consume: list[list[float]] = []
        self.circuits_stats_daily_opinion: list[list[float]] = []

        self.blocks_stats_daily_deficit: list[list[float]] = []
        self.blocks_stats_daily_demand: list[list[float]] = []
        self.blocks_stats_daily_consume: list[list[float]] = []
        self.blocks_stats_daily_opinion: list[list[float]] = []

        self.general_offer_stats: list[float] = []
        self.general_stored_energy: list[float] = []

        self.generation_stats: list[list[float]] = []
        self.is_working_thermoelectrics_stats: list[int] = []
        self.stored_energy_stats: list[list[float]] = []

        self.daily_distribution_block_thermoelectric: list[list[tuple[int, int]]] = []
        self.fitness: list[float] = []

    def recollect_thermoelectric_stats(self):

        self.general_offer_stats.append(self.world_state_manager.general_offer)

        self.generation_stats.append(
            self.world_state_manager.generation_per_thermoelectric
        )

        self.is_working_thermoelectrics_stats.append(
            [
                thermoelectric.is_working()
                for thermoelectric in self.world_state_manager.thermoelectrics
            ]
        )

        self.stored_energy_stats.append(
            [
                thermoelectric.current_capacity
                for thermoelectric in self.world_state_manager.thermoelectrics
            ]
        )

        self.general_stored_energy.append(sum(self.stored_energy_stats[-1]))

        flat_distribution, score = self.chief_agent.last_date_result

        self.daily_distribution_block_thermoelectric.append(flat_distribution)
        self.fitness.append(score)

    def recollect_circuit_block_stats(self):
        circuits_demand = []
        circuits_consume = []
        circuits_deficit = []
        circuits_opinion = []

        blocks_demand = []
        blocks_consume = []
        blocks_deficit = []
        blocks_opinion = []

        for circuit in self.world_state_manager.circuits:
            demand = 0
            consume = 0

            for block in circuit.blocks:
                report = block.history_report[-1]
                demand += report.total_demand
                consume += report.total_consumed

                blocks_demand.append(report.total_demand)
                blocks_consume.append(report.total_consumed)
                blocks_deficit.append(report.total_demand - report.total_consumed)
                blocks_opinion.append(report.citizens_opinion)

            circuits_demand.append(demand)
            circuits_consume.append(consume)
            circuits_deficit.append(demand - consume)
            circuits_opinion.append(circuit.circuit_satisfaction)  # TODO: Review this

        self.circuits_stats_daily_consume.append(circuits_consume)
        self.circuits_stats_daily_demand.append(circuits_demand)
        self.circuits_stats_daily_deficit.append(circuits_deficit)
        self.circuits_stats_daily_opinion.append(circuits_opinion)

        self.blocks_stats_daily_consume.append(blocks_consume)
        self.blocks_stats_daily_demand.append(blocks_demand)
        self.blocks_stats_daily_deficit.append(blocks_deficit)
        self.blocks_stats_daily_opinion.append(blocks_opinion)

        self.general_demand_stats.append(sum(circuits_demand))

        self.general_consume_stats.append(sum(circuits_consume))

        self.general_deficit_stats.append(
            self.general_demand_stats[-1] - self.general_consume_stats[-1]
        )

        self.general_opinion_stats.append(self.world_state_manager.general_satisfaction)

    def count_props_single_action(self, action, dict):
        for key, value in vars(action).items():
            if value:
                if key in dict:
                    dict[key] += 1
                else:
                    dict[key] = 1

    def process_thermoelectric_agent_actions(self):
        """Returns the count of active properties in the action list for each property"""
        day_i = []

        for actions_of_day in self.thermoelectric_agent_actions:
            counts = {}
            for action in actions_of_day:
                self.count_props_single_action(action=action, dict=counts)
            day_i.append(counts)
        return day_i

    def process_chief_agent_actions(self):
        counts = {}

        for action in self.chief_agent_actions:
            self.count_props_single_action(action=action, dict=counts)

        return counts

    def create_thermoelectric_agent_perception(
        self, thermoelectric
    ) -> ThermoelectricAgentPerception:

        return ThermoelectricAgentPerception(
            thermoelectric=thermoelectric,
            general_deficit=self.world_state_manager.general_deficit,
            general_demand=self.world_state_manager.general_demand,
            general_offer=self.world_state_manager.general_offer,
        )

    def create_chief_agent_perception(self):

        return ChiefElectricCompanyAgentPerception(
            thermoelectrics_id=self.world_state_manager.thermoelectrics_id,
            circuits_id=self.world_state_manager.circuits_id,
            generation_per_thermoelectric=self.world_state_manager.generation_per_thermoelectric,
            distance_matrix=self.world_state_manager.distance_matrix,
            demand_per_block_in_circuit=self.world_state_manager.predicted_demand_per_block_in_circuits,
            total_demand_per_circuit=self.world_state_manager.predicted_total_demand_per_circuit,
            circuits_importance=self.world_state_manager.circuits_importance,
            importance_per_block_in_circuits=self.world_state_manager.importance_per_block_in_circuits,
            opinion_per_block_in_circuits=self.world_state_manager.opinion_per_block_in_circuits,
            satisfaction_per_circuit=self.world_state_manager.satisfaction_per_circuit,
            industrialization_per_circuit=self.world_state_manager.industrialization_per_circuit,
            last_days_off_per_block_in_circuits=self.world_state_manager.last_days_off_per_block_in_circuits,
            longest_sequence_off_per_block_in_circuits=self.world_state_manager.longest_sequence_off_per_block_in_circuits,
            general_satisfaction=self.world_state_manager.general_satisfaction,
        )

    def update_thermoelectric(self):
        for thermoelectric in self.world_state_manager.thermoelectrics:
            thermoelectric.update()

    def update_circuits(self, opinion_day: float = False):
        for circuit in self.world_state_manager.circuits:
            circuit.update(self.world_state_manager.general_satisfaction, opinion_day)

    def simulate(self, simulation_days=50):

        start_time = time.time()

        thermoelectric_agent_rules = {
            "max_power_output": TAMaxPowerOutputDesire(),
            "minimize_downtime": TAMinimizeDowntimeDesire(),
            "prevent_unexpected_breakdowns": TAPreventUnexpectedBreakdownDesire(),
            "meet_energy_demand": TAMeetEnergyDemandDesire(),
            "prioritize_critical_parts_repair": TAPrioritizeCriticalPartsRepairDesire(),
            "repair_parts": TARepairPartsDesire(),
        }

        thermoelectric_agent_current_rules = [
            "max_power_output",
            "minimize_downtime",
            "prevent_unexpected_breakdowns",
            "meet_energy_demand",
            "prioritize_critical_parts_repair",
            "repair_parts",
        ]

        for thermoelectric in self.world_state_manager.thermoelectrics:
            agent_initial_perception = self.create_thermoelectric_agent_perception(
                thermoelectric=thermoelectric
            )

            thermoelectric_agent = ThermoelectricAgent(
                name=f"Agent of {thermoelectric.id}",
                thermoelectric=thermoelectric,
                rules=thermoelectric_agent_rules,
                current_rules=thermoelectric_agent_current_rules,
                perception=agent_initial_perception,
            )

            self.thermoelectric_agents.append(thermoelectric_agent)

        chief_agent_perception = self.create_chief_agent_perception()

        chief_agent_desires = {
            "meet_demand": CECAMeetDemandDesire(),
            "prioritize_block_importance": CECAPrioritizeBlockImportance(),
            "prioritize_block_opinion": CECAPrioritizeBlockOpinion(),
            "prioritize_consecutive_days_off": CECAPrioritizeConsecutiveDaysOff(),
            "prioritize_days_off": CECAPrioritizeDaysOff(),
        }

        chief_agent_current_desires = [
            "meet_demand",
            "prioritize_block_importance",
            "prioritize_block_opinion",
            "prioritize_consecutive_days_off",
            "prioritize_days_off",
        ]

        self.chief_agent = ChiefElectricCompanyAgent(
            name="Chief Electric Company Agent",
            thermoelectrics=self.world_state_manager.thermoelectrics,
            circuits=self.world_state_manager.circuits,
            perception=chief_agent_perception,
            rules=chief_agent_desires,
            current_rules=chief_agent_current_desires,
            mapper_key_to_circuit_block=self.world_state_manager.mapper_key_to_circuit_block,
            learn=False,
        )

        end_time = time.time()
        print(f"Initialization took {end_time - start_time} seconds")

        for day in range(simulation_days):
            start_day_time = time.time()
            print(
                f"Day {day} 🚀-----------------------------------------------------------------"
            )

            today_actions = []

            for agent_index in RANDOM.permutation(
                range(len(self.thermoelectric_agents))
            ):

                agent: ThermoelectricAgent = self.thermoelectric_agents[agent_index]
                new_agent_perception = self.create_thermoelectric_agent_perception(
                    agent.thermoelectric
                )

                agent_action_result = agent.action(new_agent_perception)
                today_actions.append(agent_action_result)
                agent.thermoelectric.update_capacity()
                self.world_state_manager.update_only_thermoelectric_stats(agent_index)

            self.thermoelectric_agent_actions.append(today_actions)
            # distribute energy

            self.chief_agent.learn = day % int(10 / 3) == 0
            chief_agent_perception = self.create_chief_agent_perception()
            chief_agent_action = self.chief_agent.action(chief_agent_perception)
            self.chief_agent_actions.append(chief_agent_action)

            self.update_circuits(opinion_day=(day % 30 == 0))

            self.recollect_thermoelectric_stats()

            self.update_thermoelectric()

            self.world_state_manager.update()

            self.recollect_circuit_block_stats()

            end_day_time = time.time()
            print(f"Day {day} took {end_day_time - start_day_time} seconds")

        # self.plot_stats()


simulation = Simulation(
    circuits=electric_system_circuits_list,
    thermoelectrics=electric_system_thermoelectrics_list,
    distance_matrix=matrix,
    get_circuit_importance=get_circuit_importance,
    get_block_importance=get_block_importance,
)

simulation.simulate()

Initialization took 0.0 seconds
Day 0 🚀-----------------------------------------------------------------
Imprimiendo GENERATED DESIRE 
        id: generated_desire_5.
        description : IF tightly covered system AND good general opinion AND many thermoelectrics working THEN meet_demand AND max_stored_energy,
        weight: 1.
        
generación [20108.004933571854, 11996.362329343749, 3763.6444700350594, 13842.631289915673, 3248.7501476002417, 6439.231090654619, 9358.51697436783, 15009.566096312617]
asignado [17564.852978836334, 10379.23414341751, 3228.336032968675, 12157.808296169194, 2820.183181023183, 5564.400175320315, 8182.977076583603, 13077.048852483005]
planificado [20105.788545657637, 11995.290980392367, 3763.5556044964446, 13841.03428356354, 3247.69554853083, 6435.1026894543065, 9357.536719610745, 15004.906552747125]


RuntimeError: CASO DE PRUEBA GENERADO...

# Estadísticas

In [13]:
import pickle

### Do something using simulation instance

# Plotting general stats for deficit, consume, and demand per day
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=list(range(len(simulation.general_demand_stats))),
        y=simulation.general_demand_stats,
        mode="lines",
        name="Demand",
    )
)

fig.add_trace(
    go.Scatter(
        x=list(range(len(simulation.general_stored_energy))),
        y=simulation.general_stored_energy,
        mode="lines",
        name="Stored Energy",
    )
)

fig.add_trace(
    go.Scatter(
        x=list(range(len(simulation.general_consume_stats))),
        y=simulation.general_consume_stats,
        mode="lines",
        name="Consume",
    )
)
fig.add_trace(
    go.Scatter(
        x=list(range(len(simulation.general_deficit_stats))),
        y=simulation.general_deficit_stats,
        mode="lines",
        name="Deficit",
    )
)
fig.update_layout(
    title="Daily Deficit, Consume, and Demand",
    xaxis_title="Day",
    yaxis_title="Value",
    height=800,
)

fig.show()


fig_working_thermoelectrics = go.Figure()
fig_working_thermoelectrics.add_trace(
    go.Scatter(
        x=list(range(len(simulation.is_working_thermoelectrics_stats))),
        y=[sum(day) for day in simulation.is_working_thermoelectrics_stats],
        mode="lines",
        name="Working Thermoelectrics",
    )
)
fig_working_thermoelectrics.update_layout(
    title="Working Thermoelectrics per Day",
    xaxis_title="Day",
    yaxis_title="Number of Working Thermoelectrics",
    height=800,
)

fig_working_thermoelectrics.show()


fig_generation = go.Figure()

for th_index in range(len(simulation.world_state_manager.thermoelectrics)):
    fig_generation.add_trace(
        go.Scatter(
            x=list(range(len(simulation.generation_stats))),
            y=[day[th_index] for day in simulation.generation_stats],
            mode="lines",
            name=f"Thermoelectric {th_index}",
        )
    )


fig_generation.show()


fig_offer = go.Figure()
fig_offer.add_trace(
    go.Scatter(
        x=list(range(len(simulation.general_offer_stats))),
        y=simulation.general_offer_stats,
        mode="lines",
        name="Offer",
    )
)
fig_offer.update_layout(
    title="Daily Offer", xaxis_title="Day", yaxis_title="Value", height=800
)

fig_offer.show()

# chief actions
dias = list(range(len(simulation.chief_agent_actions)))
propiedades = list(vars(simulation.chief_agent_actions[0]).keys())

data_matrix = []
for i in dias:
    row = [
        1 if getattr(simulation.chief_agent_actions[i], prop) else 0
        for prop in propiedades
    ]
    data_matrix.append(row)

fig = go.Figure()

for i, prop in enumerate(propiedades):
    fig.add_trace(
        go.Scatter(
            x=dias, y=[row[i] for row in data_matrix], mode="lines+markers", name=prop
        )
    )

fig.update_layout(
    title="Chief Actions by Day",
    xaxis_title="Day",
    yaxis_title="Active (1) / Inactive (0)",
    height=800,
)

fig.show()


# thermoelectric agent actions


day_i = simulation.process_thermoelectric_agent_actions()

propiedades = list(vars(simulation.thermoelectric_agent_actions[0][0]).keys())

dias = list(range(len(day_i)))

fig = go.Figure()

for prop in propiedades:
    fig.add_trace(
        go.Scatter(
            x=dias,
            y=[day.get(prop, 0) for day in day_i],
            mode="lines+markers",
            name=prop,
        )
    )

fig.update_layout(
    title="Thermoelectric Agent Actions by Day",
    xaxis_title="Day",
    yaxis_title="Count of Actions",
    height=800,
)

fig.show()


class SimulationCloneDumperClass:
    def __init__(self, simulation: Simulation):
        self.world_state_manager = simulation.world_state_manager
        self.thermoelectric_agents = simulation.thermoelectric_agents
        self.thermoelectric_agent_actions = simulation.thermoelectric_agent_actions
        # self.chief_agent = simulation.chief_agent
        self.chief_agent_actions = simulation.chief_agent_actions

        # STATS
        self.general_deficit_stats = simulation.general_deficit_stats
        self.general_consume_stats = simulation.general_consume_stats
        self.general_demand_stats = simulation.general_demand_stats
        self.general_opinion_stats = simulation.general_opinion_stats

        self.circuits_stats_daily_deficit = simulation.circuits_stats_daily_deficit
        self.circuits_stats_daily_demand = simulation.circuits_stats_daily_demand
        self.circuits_stats_daily_consume = simulation.circuits_stats_daily_consume
        self.circuits_stats_daily_opinion = simulation.circuits_stats_daily_opinion

        self.blocks_stats_daily_deficit = simulation.blocks_stats_daily_deficit
        self.blocks_stats_daily_demand = simulation.blocks_stats_daily_demand
        self.blocks_stats_daily_consume = simulation.blocks_stats_daily_consume
        self.blocks_stats_daily_opinion = simulation.blocks_stats_daily_opinion

        self.general_offer_stats = simulation.general_offer_stats
        self.general_stored_energy = simulation.general_stored_energy

        self.generation_stats = simulation.generation_stats
        self.is_working_thermoelectrics_stats = (
            simulation.is_working_thermoelectrics_stats
        )
        self.stored_energy_stats = simulation.stored_energy_stats

        self.daily_distribution_block_thermoelectric = (
            simulation.daily_distribution_block_thermoelectric
        )
        self.fitness = simulation.fitness


simulationDumper = SimulationCloneDumperClass(simulation=simulation)
with open("simulation_instance.pkl", "wb") as file:
    pickle.dump(simulationDumper, file)